# Стратегии генерации


(based on https://github.com/huggingface/blog/blob/main/notebooks/02_how_to_generate.ipynb)

### **Введение**

В последние годы растет интерес к языковой генерации благодаря появлению больших языковых моделей на основе трансформеров, обучаемых на миллионах веб-страниц, таких как знаменитая OpenAI [GPT2 model](https://openai.com/blog/better-language-models/). Результаты впечатляют, e.g. [GPT2 on unicorns](https://openai.com/blog/better-language-models/#samples), [XLNet](https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e), [Controlled language with CTRL](https://blog.einstein.ai/introducing-a-conditional-transformer-language-model-for-controllable-generation/). Помимо улучшенной архитектуры transformer и большого объема обучающих данных, важную роль также сыграли улучшенные методы декодирования.

В этом блокноте дается краткий обзор различных стратегий декодирования и, что более важно, показано, как * вы * можете реализовать их с минимальными усилиями, используя популярную библиотеку `transformers`!

Все следующие функциональные возможности могут быть использованы для **авторегрессионной** генерации языка ([тут](http://jalammar.github.io/illustrated-gpt2/) как это делается). Если коротко, *авторегрессионная* генерация языка основана на предположении, что распределение вероятностей последовательности слов может быть разложено на произведение следующих условных вероятностей:
$$ P(w_{1:T} | W_0 ) = \prod_{t=1}^T P(w_{t} | w_{1: t-1}, W_0) \text{ ,with }  w_{1: 0} = \emptyset, $$


Авторегрессивная генерация языка теперь доступна для `GPT2`, `XLNet`, `OpenAi-GPT`, `CTRL`, `TransfoXL`, `XLM`, `Bart`, `T5`!

Мы проведем экскурсию по наиболее известным в настоящее время методам декодирования, главным образом *Greedy search*, *Beam search*, *Top-K sampling*  и *Top-p sampling*.


In [ ]:
#!pip install -q git+https://github.com/huggingface/transformers.git  # the bleeding edge version 
#!pip install -q tensorflow==2.1
!pip install transformers==4.18.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 33.3 MB/s 
     |████████████████████████████████| 182 kB 56.8 MB/s 
     |████████████████████████████████| 6.6 MB 30.0 MB/s 
     |████████████████████████████████| 880 kB 13.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=9ac6948671ff96bc858f18478331ba20bdfc1d7e3ef42150151186ce34f40203
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
import torch

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

### **Greedy Search**

Жадный поиск просто выбирает слово с наибольшей вероятностью в качестве следующего слова: $w_t = argmax_{w}P(w | w_{1:t-1})$ на каждом временном шаге $t$. На следующем рисунке показан Greedy Search.

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)

Начиная со слова $\text{"The"}$, алгоритм
жадно выбирает следующее слово с наибольшей вероятностью $\text{"nice"}$ и так далее, так что конечная сгенерированная последовательность слов будет $\text{"The", "nice", "woman"}$ с общей вероятностью $0.5 \times 0.4 = 0.2$.

В дальнейшем мы будем генерировать последовательности слов, используя GPT2 в контексте $(\text{"I", "enjoy", "walking", "with", "my", "cute", "dog"})$. Давайте посмотрим, как можно использовать жадный поиск в "трансформерах" следующим образом
:

In [ ]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='pt')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll


Хорошо! Мы сгенерировали наш первый короткий текст с помощью GPT2 😊. Сгенерированные слова, следующие контексту, разумны, но модель быстро начинает повторяться! Это очень распространенная проблема при генерации языка в целом и, по-видимому, еще более распространена при grid / beam search'e - проверьте [Vijayakumar et al., 2016](https://arxiv.org/abs/1610.02424) and [Shao et al., 2017](https://arxiv.org/abs/1701.03185).

Однако основным недостатком жадного поиска является то, что он пропускает слова с высокой вероятностью, скрытые за словом с низкой вероятностью, как видно из нашего наброска выше:

Слово $\text{"has"}$ с его высокой условной вероятностью $0,9$ скрыто за словом $\text{"dog"}$, которое имеет только вторую по величине условную вероятность, так что жадный поиск пропускает последовательность слов $\text{"The"}, \text{"dog"}, \text{"has"}$.

К счастью, у нас есть beam search, чтобы решить эту проблему!


### **Beam search**

Поиск по лучу (beam search) снижает риск пропуска скрытых последовательностей слов с высокой вероятностью, сохраняя наиболее вероятные "num_beams" гипотез на каждом временном шаге и в конечном итоге выбирая гипотезу, имеющую в целом наибольшую вероятность. Давайте проиллюстрируем это с помощью `num_beams=2`:

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)

На временном шаге $1$, помимо наиболее вероятной гипотезы $\text{"The", "woman"}$, beam search также отслеживает вторую наиболее вероятную гипотезу $\text{"The", "dog"}$. На временном шаге $2$ поиск по лучу обнаруживает, что последовательность слов $\text{"The", "dog", "has"}$ имеет вероятность $0,36$ более высокую, чем $\text{"The", "nice", "woman"}$, которая имеет $0,2$. Отлично, он нашел наиболее вероятную последовательность слов в нашем примере!

Поиск по лучу всегда найдет выходную последовательность с большей вероятностью, чем жадный поиск, но не гарантирует нахождения наиболее вероятного результата. 

Давайте посмотрим, как поиск по лучу можно использовать в "трансформерах". Мы устанавливаем `num_beams > 1` и `early_stopping=True`, чтобы генерация завершалась, когда все гипотезы луча достигали токена EOS.

In [ ]:
# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,  
    max_length=50, 
    num_beams=5, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I'm not sure if I'll ever be able to walk with him again. I'm not sure if I'll


Хотя результат, возможно, более плавный, выход по-прежнему включает повторения одних и тех же последовательностей слов.  
Простое решение состоит в том, чтобы ввести штрафы за *n-граммы* (*также известные как последовательности слов из $n$ слов), как это было введено [Paulus et al. (2017)](https://arxiv.org/abs/1705.04304 ) и [Klein et al. (2017)](https://arxiv.org/abs/1701.02810 ). Наиболее распространенный штраф *n-граммов* гарантирует, что ни один *n-грамм* не появится дважды, вручную устанавливая вероятность следующих слов, которые могли бы создать уже встречавшийся *n-грамм*, равной $0$.

Давайте попробуем это, установив `no_repeat_n gram_size=2`, чтобы ни один *2-gramm* не появлялся дважды:

In [ ]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break


Мило, это выглядит намного лучше! Мы можем видеть, что повторение больше не появляется. Тем не менее, *n-граммовые* штрафы следует использовать с осторожностью. В статье, сгенерированной о городе *Нью-Йорк*, не следует использовать штраф в размере *2-gramms* или название города будет встречаться только один раз во всем тексте!

Еще одной важной особенностью поиска луча является то, что мы можем сравнить верхние лучи после генерации и выбрать сгенерированный луч, который лучше всего соответствует нашей цели. 

В `transformers` мы просто устанавливаем параметр `num_return_sequences` на количество лучей с наибольшим количеством баллов, которые должны быть возвращены. Убедитесь, однако, что `num_return_sequences <= num_beams`!

In [ ]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}\n".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break

1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to get back to

2: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to take a break

3: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to get back to

4: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking ab

## **Sampling**

В своей самой простой форме sampling означает случайный выбор следующего слова $w_t$ в соответствии с его условным распределением вероятностей:

$$w_t \sim P(w|w_{1:t-1})$$

Используя приведенный выше пример, на следующем рисунке визуализируется генерация языка при выборке.

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

Становится очевидным, что генерация языка с использованием sampling больше не является "детерминированной". Cлово 
$\text{"car"}$ выбирается из условного распределения вероятностей $P(w |\text{"The"})$, за которым следует sampling $\text{"drives"}$ из $P(w |\text{"The"}, \text{"car"})$.

В `transformers` мы устанавливаем `do_sample=True` и деактивируем sampling *Top-K* (подробнее об этом позже) через `top_k=0`. Не стесняйтесь изменять `random_seed`, чтобы поиграть с моделью.

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(1)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog up playground. It's so happy seeing humans happy, I totally get it. I love it when everyone from the park to the state dislikes carving up campsites and going and attacking people trying to get their dogs


Интересно! Текст кажется нормальным, но при ближайшем рассмотрении оказывается, что он не очень связный. Слова *new hand sense* и *local batte harness* очень странные и звучат не так, как будто они были написаны человеком. Это большая проблема при выборке последовательностей слов: модели часто генерируют бессвязную тарабарщину, *ср.* [Ари Хольцман и др. (2019)](https://arxiv.org/abs/1904.09751 ).

Хитрость заключается в том, чтобы сделать распределение $P(w|w_{1:t-1})$ более четким (увеличивая вероятность слов с высокой вероятностью и уменьшая вероятность слов с низкой вероятностью), понизив так называемую "температуру" [softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max ). 

$q = \frac{exp(z_i / T)}{\sum_j exp(z_j / T)}$

Иллюстрация применения температуры к нашему примеру, приведенному выше, может выглядеть следующим образом.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

Условное распределение следующего слова на шаге $t = 1$ становится намного более четким, практически не оставляя шансов для выбора слова $\text{"car"}$.


In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(0)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=0, 
    temperature=0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog along the trail. He's taken my dog home for the week and that's been a fantastic experience."

Plus, the shearer has been given a small sailboat and he can sail through the Quarry


## **Top-K Sampling**

[Fan et. al (2018)](https://arxiv.org/pdf/1805.04833.pdf ) ввел простую, но очень мощную схему выборки, называемую выборкой ***Top-K***. В выборке *Top-K* фильтруются *K* наиболее вероятных следующих слов, и масса вероятности перераспределяется только между этими *K* следующими словами. 
GPT2 принял эту схему выборки, что стало одной из причин его успеха. 

Мы расширили диапазон слов, используемых для обоих этапов выборки в приведенном выше примере, с 3 слов до 10 слов, чтобы лучше проиллюстрировать выборку *Top-K*.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)

Установив $K = 6$, на обоих этапах выборки мы ограничиваем наш пул выборок 6 словами. В то время как 6 наиболее вероятных слов, определенных как $V_ {\text{top-K}}$, охватывают только *около* двух третей всей массы вероятности на первом шаге, они включают почти всю массу вероятности на втором шаге. Тем не менее, мы видим, что он успешно устраняет довольно странных кандидатов $\text{"not", "the", "small", "told"}$ 
на втором этапе отбора проб.


Давайте посмотрим, как *Top-K* можно использовать в библиотеке, установив `top_k=50`:

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(0)

# set top_k to 50
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but it's pretty much just my usual dog routine.

In the evening I have one of those little dog-loving cat ladies playing in the backyard.

And with our kids there I often get


Совсем неплохо! Этот текст, пожалуй, самый "человечный" на сегодняшний день. 
Однако одна проблема с выборкой *Top-K* заключается в том, что она динамически не адаптирует количество слов, которые отфильтровываются из распределения вероятности следующего слова $P (w |w_ {1:t-1})$.
Это может быть проблематично, поскольку некоторые слова могут быть выбраны из очень четкого распределения (распределение справа на графике выше), в то время как другие - из гораздо более плоского распределения (распределение слева на графике выше). 

На шаге $t=1$, *Top-K* исключает возможность 
примеров $\text{"people", "big", "house", "cat"}$, которые кажутся разумными кандидатами. С другой стороны, на шаге $t= 2$ метод включает, возможно, неправильно подобранные слова $\text{"down", "a"}$ в выборку слов. Таким образом, ограничение пула выборок фиксированным размером *K* может привести к тому, что модель будет выдавать тарабарщину для четких распределений и ограничит творческий потенциал модели для плоского распределения.
Эта интуиция привела [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751 ) для создания ***Top-p*** - или ***nucleus***-sampling.

## **Sampling Top-p (nucleus)**

Вместо выборки только из наиболее вероятных *K* слов, в выборке *Top-p* выбирается из наименьшего возможного набора слов, совокупная вероятность которых превышает вероятность *p*. Затем масса вероятности перераспределяется между этим набором слов. Таким образом, размер набора слов (количество слов в наборе) может динамически увеличиваться и уменьшаться в соответствии с распределением вероятности следующего слова. Ладно, это было очень многословно, давайте визуализируем.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

Установив $p=0.92$, выборка *Top-p* выбирает *минимальное* количество слов, которое в совокупности должно превышать $p=92\%$ от массы вероятности, определяемой как $V_{\text{top-p}}$. В первом примере это включало 9 наиболее вероятных слов, тогда как во втором примере нужно выбрать только 3 лучших слова, чтобы их вероятностная масса превысила 92%. На самом деле все очень просто! Можно видеть, что стратегия сохраняет широкий диапазон слов, где следующее слово, возможно, менее предсказуемо, *например* $P(w | \text{"The"})$, и только несколько слов, когда следующее слово кажется более предсказуемым, *например* $P(w | \text{"The", "car"})$.

Мы активируем выборку *Top-p*, установив `0 < top_p < 1`:

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(3)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_p=0.92, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog.

Meet Szybras:

Szybras has been married seven years and is a chronic participant in church and outdoors activities. The family has been looking for a home and she has been


Отлично, звучит так, словно это мог бы написать человек. Ну, может быть, еще не совсем. 

Хотя в теории *Top-p* кажется более элегантным, чем *Top-K*, оба метода хорошо работают на практике. *Top-p* также можно использовать в сочетании с *Top-K*, что позволяет избежать слов с очень низким скором, обеспечивая при этом некоторый динамический выбор.

Наконец, чтобы получить несколько независимо отобранных выходных данных, мы можем установить параметр `num_return_sequences > 1`:

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(0)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, and she is the perfect animal companion to me. She helps me get on my feet as often as possible. I love sharing my love and care with all of my dogs. I have the highest respect for them
1: I enjoy walking with my cute dog when he is around my neck," she said. "I'm just doing it. It's not something that's easy for me to do when I'm the leader."

The family is just beginning to see
2: I enjoy walking with my cute dog, and we are both very pleased by his behavior. He seems to be extremely curious about the world around him, and just as he is searching for his place of origin he is able to spot and find it easily


## Constrained generation

Для лучшей читаемости мы будем использовать end-of-line в качестве токена EOS, вместо того, чтобы просто всегда генерировать 50 токенов.

In [ ]:
END_OF_LINE = tokenizer('\n').input_ids[0]
print(END_OF_LINE)

198


Поиск по базовому лучу выдает очень похожие предложения; все они содержат "not sure" или "don't think".

In [ ]:
beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True,
    eos_token_id=END_OF_LINE,
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

2: I enjoy walking with my cute dog, but I don't think I'll ever be able to walk with her again.

3: I enjoy walking with my cute dog, but I don't think I'll ever be able to walk with him again.

4: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again. I don't know what to do.



Что произойдет, если мы запретим модели использовать эти фразы?



In [ ]:
beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True,
    eos_token_id=END_OF_LINE,
    bad_words_ids=tokenizer(['sure', 'think', 'thundersnatch'], add_prefix_space=True)['input_ids'],
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I don't like to walk alone.

1: I enjoy walking with my cute dog, but I don't want to have to go through the hassle of going to the vet.

2: I enjoy walking with my cute dog, but I don't want to have to go through the hassle of going to the vet to get a new dog.

3: I enjoy walking with my cute dog, but I don't want to have to go through the hassle of going to the vet to get a new one.

4: I enjoy walking with my cute dog, but I don't want to have to go through the hassle of going to the vet to see if my dog is sick.



In [ ]:
tokenizer(['sure', ' sure', ' I am not sure'])['input_ids']

[[19532], [1654], [314, 716, 407, 1654]]

Мы видим, что смысл этих текстов сильно изменился – но каким-то непредсказуемым образом. 

Можем ли мы заставить модель написать текст с участием "cat"?

In [ ]:
beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True,
    eos_token_id=END_OF_LINE,
    bad_words_ids=tokenizer(['sure', 'think'], add_prefix_space=True)['input_ids'],
    force_words_ids=[tokenizer(['cat'], add_prefix_space=True, add_special_tokens=False).input_ids],
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not a cat person.

1: I enjoy walking with my cute dog, but I'm not a cat person."

2: I enjoy walking with my cute dog, but I'm not a cat person, so I don't know what to do with him."

3: I enjoy walking with my cute dog, but I'm not a cat person, so I don't know what to do with him. He's my best friend."

4: I enjoy walking with my cute dog, but I'm not a cat person, so I don't know what to do with him. He's my best friend.



Уточнение: **force_words_ids** - это список ограничений. Каждое ограничение представляет собой список выражений, таких, что по крайней мере одно выражение должно быть включено в сгенерированный текст. И каждое выражение - это просто список токенов. 

Смотрите обсуждение в [the HF pull request](https://github.com/huggingface/transformers/issues/14081 ), или прочтите статью "[Guided Generation of Cause and Effect](https://www.ijcai.org/proceedings/2020/0502.pdf )" Ли и др., где был предложен алгоритм.  

Чтобы оценить силу этих ограничений, давайте заставим модель включить мышь (или даже много мышей) в текст. Мы также можем ослабить ограничение "кошка", разрешив вместо него использовать слова "кошки", "котенок" или "кошачий".

In [ ]:
beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True,
    eos_token_id=END_OF_LINE,
    bad_words_ids=tokenizer(['sure', 'think'], add_prefix_space=True)['input_ids'],
    force_words_ids = [
        tokenizer(['cat', 'cats', 'kitten', 'feline', 'Cat', 'Cats'], add_prefix_space=True, add_special_tokens=False).input_ids,
        tokenizer(['mouse', 'mice'], add_prefix_space=True, add_special_tokens=False).input_ids,
    ],
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog. I love feline companionship and I like mice."

1: I enjoy walking with my cute dog. I love feline companionship and I like mice.

2: I enjoy walking with my cute dog. I love feline companionship and I like mice and cats.

3: I enjoy walking with my cute dog. I love feline companionship and I like to mouse my way around the house.

4: I enjoy walking with my cute dog. I love feline companionship and I like to mouse my way through the world.



## **Заключение**

Методы декодирования *ad-hoc*, выборка *top-p* и *top-K*, по-видимому, обеспечивают более плавный текст, чем традиционный *жадный* и *лучевой* поиск при генерации открытого языка. 
Однако в последнее время появилось больше доказательств того, что очевидные недостатки *жадного* и *лучевого* поиска - в основном генерирующие повторяющиеся последовательности слов - вызваны моделью (особенно способом обучения модели), а не методом декодирования,  [Уэллек и др. (2019)](https://arxiv.org/pdf/1908.04319.pdf ). Кроме того, как показано в [Welleck et al. (2020)](https://arxiv.org/abs/2002.02492 ), похоже, что выборки *top-K* и *top-p* также страдают от генерации повторяющихся последовательностей слов.

В [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf ), авторы показывают, что, согласно человеческим оценкам, поиск *beam* может генерировать более плавный текст, чем выборка *Top-p*, при адаптации цели обучения модели.

Генерация открытого языка - быстро развивающаяся область исследований, и, как это часто бывает, здесь нет универсального метода, поэтому нужно посмотреть, что лучше всего работает в конкретном случае использования.

Хорошо, что *вы* можете опробовать все различные методы декодирования в "трансфомерах" 🤗. 

Это было краткое введение о том, как использовать различные методы декодирования в "трансформерах", и о последних тенденциях в создании языков с открытым исходным кодом. 

## **Приложение**

Есть пара дополнительных параметров для метода `generate`, которые не были упомянуты выше. Мы кратко объясним их здесь!

- `min_length` можно использовать, чтобы заставить модель не выдавать токен EOS (= не заканчивать предложение) до достижения `min_length`. Это довольно часто используется при суммаризации, но может быть полезно в целом, если пользователь хочет получить более длинные выходные данные.
- `repetition_penalty` может использоваться для "наказания" слов, которые уже были сгенерированы или принадлежат контексту. Впервые он был представлен [Kesker et al. (2019)](https://arxiv.org/abs/1909.05858 ) и также используется в цели обучения в [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf ). Это может быть довольно эффективно для предотвращения повторений, но, по-видимому, очень чувствительно к различным моделям и вариантам использования, *например* смотрите это [обсуждение](https://github.com/huggingface/transformers/pull/2303 ) на Github.

- `attention_mask` можно использовать для маскировки дополненных токенов
- `pad_token_id`, `bos_token_id`, `eos_token_id`: Если в модели по умолчанию нет этих токенов, пользователь может вручную выбрать другие идентификаторы токенов для их представления.
